IMPORTACIONES Y CONFIGURACION INICIAL

In [1]:
import os
import json
import gc
import random
import numpy as np
import pandas as pd
import optuna
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, balanced_accuracy_score
import matplotlib.pyplot as plt
from optuna.visualization import plot_optimization_history

# --- Configuración de Semillas para Reproducibilidad ---
SEED = 43
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# --- Constantes ---
SCORE = "f1-score"  # Métrica para optimizar en Optuna
TIMESTEPS = 5      # Número de días pasados para usar como una secuencia
MODEL_NAME = 'LSTM_RNN'

/home/toyotx22/bitcoin_sentiment_forecaster/tf_venv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-08-29 12:53:16.794291: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-29 12:53:16.794425: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-29 12:53:16.809614: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-29 12:53:16.856738: I tensorflow/core/platform/cpu_feature_guard.cc:18

FUNCIONES DE METRICAS

In [2]:
def get_trend_changes_report_dict(y_test: np.array, y_pred: np.array) -> float:
    """
    Calculate the trend changes score based on the test and predicted values.
    
    Args:
        y_test (np.array): True labels.
        y_pred (np.array): Predicted labels.
        
    Returns:
        float: The trend changes score.
    """
    y_df = pd.DataFrame([y_test, y_pred]).T
    y_df.columns = ["y_test", "y_pred"]
    y_df["y_test_shifted"] = y_df["y_test"].shift(-1)
    y_df["is_changed_trend_test"] = y_df["y_test"] != y_df["y_test_shifted"]
    y_df["y_predict_shifted"] = y_df["y_pred"].shift(-1)
    y_df["is_changed_trend_predict"] = y_df["y_pred"] != y_df["y_predict_shifted"]
    return classification_report(
        y_df["is_changed_trend_test"][:-1], 
        y_df["is_changed_trend_predict"][:-1], 
        digits=4,
        output_dict=True,
        zero_division=0
    )

def trend_changes_score(y_test: np.array, y_pred: np.array) -> float:
    """
    Calculate the trend changes score based on the test and predicted values.
    
    Args:
        y_test (np.array): True labels.
        y_pred (np.array): Predicted labels.
        
    Returns:
        float: The trend changes score.
    """
    y_df = pd.DataFrame([y_test, y_pred]).T
    y_df.columns = ["y_test", "y_pred"]
    y_df["y_test_shifted"] = y_df["y_test"].shift(-1)
    y_df["is_changed_trend_test"] = y_df["y_test"] != y_df["y_test_shifted"]
    y_df["y_predict_shifted"] = y_df["y_pred"].shift(-1)
    y_df["is_changed_trend_predict"] = y_df["y_pred"] != y_df["y_predict_shifted"]
    return classification_report(
        y_df["is_changed_trend_test"][:-1], 
        y_df["is_changed_trend_predict"][:-1], 
        digits=4
    )

def trend_changes_true(y_test: np.array, y_pred: np.array) -> float:
    """
    Calculate the trend changes score based on the test and predicted values.
    
    Args:
        y_test (np.array): True labels.
        y_pred (np.array): Predicted labels.
        
    Returns:
        float: The trend changes score.
    """
    y_df = pd.DataFrame([y_test, y_pred]).T
    y_df.columns = ["y_test", "y_pred"]
    y_df["y_test_shifted"] = y_df["y_test"].shift(-1)
    y_df["is_changed_trend_test"] = y_df["y_test"] != y_df["y_test_shifted"]
    y_df["y_predict_shifted"] = y_df["y_pred"].shift(-1)
    y_df["is_changed_trend_predict"] = y_df["y_pred"] != y_df["y_predict_shifted"]
    report = classification_report(
        y_df["is_changed_trend_test"][:-1],
        y_df["is_changed_trend_predict"][:-1],
        output_dict=True,
        zero_division=0
    )
    return report["True"][SCORE]

CARGA Y PREPARACION DE DATOS

In [3]:
# --- Carga y Preparación de Datos ---

# 1. Cargar los datasets pre-divididos
training_set = pd.read_csv('../../../data/post_cleaning/training_set.csv', parse_dates=['date'])
validation_set = pd.read_csv('../../../data/post_cleaning/validation_set.csv', parse_dates=['date'])
test_set = pd.read_csv('../../../data/post_cleaning/test_set.csv', parse_dates=['date'])

# 2. Separar características (X) y objetivo (y) para cada set
X_train_df = training_set.drop(columns=['date', 'target_trend'])
y_train_df = training_set['target_trend']

X_val_df = validation_set.drop(columns=['date', 'target_trend'])
y_val_df = validation_set['target_trend']

X_test_df = test_set.drop(columns=['date', 'target_trend'])
y_test_df = test_set['target_trend']

# 3. Función para crear secuencias (la misma que tenías)
def create_sequences(features, target, time_steps=10):
    X_seq, y_seq = [], []
    for i in range(len(features) - time_steps):
        X_seq.append(features[i:(i + time_steps)])
        y_seq.append(target[i + time_steps])
    return np.array(X_seq), np.array(y_seq)

# 4. Crear secuencias POR SEPARADO para cada conjunto para evitar fugas
# Esto es el cambio clave.
X_train, y_train_seq = create_sequences(X_train_df.values, y_train_df.values, time_steps=TIMESTEPS)
X_val, y_val_seq = create_sequences(X_val_df.values, y_val_df.values, time_steps=TIMESTEPS)
X_test, y_test_seq = create_sequences(X_test_df.values, y_test_df.values, time_steps=TIMESTEPS)

# 5. Mapear clases del objetivo a 0, 1, 2
cls_map = {-1: 0, 0: 1, 1: 2}
y_train = np.vectorize(cls_map.get)(y_train_seq)
y_val = np.vectorize(cls_map.get)(y_val_seq)
y_test = np.vectorize(cls_map.get)(y_test_seq)

# (Opcional) Escalar características después de crear secuencias si lo necesitas
# scaler = MinMaxScaler()
# X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
# X_val = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)
# X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

print(f"Forma de X_train: {X_train.shape}")
print(f"Forma de y_train: {y_train.shape}")
print(f"Forma de X_val: {X_val.shape}")
print(f"Forma de y_val: {y_val.shape}")
print(f"Forma de X_test: {X_test.shape}")
print(f"Forma de y_test: {y_test.shape}")

Forma de X_train: (1369, 5, 60)
Forma de y_train: (1369,)
Forma de X_val: (289, 5, 60)
Forma de y_val: (289,)
Forma de X_test: (290, 5, 60)
Forma de y_test: (290,)


In [ ]:
# ========= Definición de modelo =========
def build_model(trial, timesteps, features):
    lstm_layers = trial.suggest_int("lstm_layers", 1, 2)
    units1 = trial.suggest_int("units1", 32, 256, step=32) 
    if lstm_layers == 2:
        # Solo sugiere 'units2' si se va a usar una segunda capa
        units2 = trial.suggest_int("units2", 32, 256, step=32)
    else:
        # Si no, 'units2' no es un hiperparámetro para este trial
        units2 = None
    dense_units = trial.suggest_int("dense_units", 16, 256, step=16) 
    dense_activation = trial.suggest_categorical("dense_activation", ["relu", "tanh"]) 
    dropout = trial.suggest_float("dropout", 0.0, 0.60) 
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-2, log=True) 
    optimizer_name = trial.suggest_categorical("optimizer", ["adam", "rmsprop", "adamw"]) 
    optimizer_map = {"adam": tf.keras.optimizers.Adam,
                 "rmsprop": tf.keras.optimizers.RMSprop,
                 "adamw": tf.keras.optimizers.AdamW}
    optimizer = optimizer_map[optimizer_name](learning_rate=learning_rate)

    model = Sequential()
    model.add(Input(shape=(timesteps, features)))

    if lstm_layers == 2:
        model.add(LSTM(units1, return_sequences=True, dropout=dropout))
        model.add(LSTM(units2, dropout=dropout))
    else:
        model.add(LSTM(units1, dropout=dropout))
    # Capas Dense
    model.add(Dropout(dropout))
    model.add(Dense(dense_units, activation=dense_activation))
    model.add(Dropout(dropout))
    model.add(Dense(3, activation="linear"))  # logits
    
    model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=optimizer,
        metrics=["accuracy"]
    )
    return model

Optimización con Optuna

In [ ]:
# ========= Objetivo Optuna =========
def objective(trial):
    # Limpieza y semillas al inicio de cada trial
    tf.keras.backend.clear_session()
    tf.random.set_seed(SEED)
    np.random.seed(SEED)
    random.seed(SEED)
 
    model = build_model(trial, X_train.shape[1], X_train.shape[2])
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    patience = trial.suggest_int("patience", 5, 20)

    es = EarlyStopping(monitor="val_loss", patience=patience, restore_best_weights=True, verbose=0)

    model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=80,
        batch_size=batch_size,
        callbacks=[es],
        verbose=0
    )

    # Predicción y métrica personalizada
    val_logits = model.predict(X_val, verbose=0)
    y_val_pred = np.argmax(val_logits, axis=1)
    score = trend_changes_true(y_val, y_val_pred)

    # Limpieza
    tf.keras.backend.clear_session()
    del model
    gc.collect()

    return score

# ========= Ejecutar Optuna =========
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=SEED))
study.optimize(objective, n_trials=200)
print("Mejores hiperparámetros:", study.best_params)
print(f"Mejor score {SCORE}: {study.best_value:.4f}")

In [ ]:
# --- Visualizar y Guardar Resultados de Optuna ---
plot_optimization_history(study)

In [ ]:
print("Mejores hiperparámetros encontrados:")
best_params = study.best_params
print(best_params)
print(f"Mejor score de '{SCORE}': {study.best_value:.4f}")

Mejores hiperparámetros encontrados:
{'lstm_layers': 1, 'units1': 192, 'dense_units': 192, 'dense_activation': 'tanh', 'dropout': 0.0003689957437176851, 'learning_rate': 2.2225431000607906e-05, 'optimizer': 'adam', 'batch_size': 32, 'patience': 6}
Mejor score de 'f1-score': 0.3636


In [ ]:
# --- Celda Nueva: Análisis de Hiperparámetros ---
from optuna.visualization import plot_param_importances, plot_slice

# 1. Gráfico de Importancia de Hiperparámetros
# Muestra qué hiperparámetros tuvieron el mayor impacto en el score.
param_importances = plot_param_importances(study)
param_importances.show()

# 2. Gráfico de Corte (Slice Plot)
# Muestra cómo varía el score para cada valor de cada hiperparámetro.
# Es excelente para ver los "rangos buenos" de cada parámetro.
slice_plot = plot_slice(study)
slice_plot.show()

Resultados y Guardado de Hiperparámetros

In [10]:
# --- Guardar en JSON ---
history_file = "best_hyperparams_lstm.json"
history_data = []
if os.path.exists(history_file):
    try:
        with open(history_file, "r") as f:
            history_data = json.load(f)
    except (json.JSONDecodeError, ValueError):
        history_data = []

history_data.append({"params": best_params, "value": study.best_value})

with open(history_file, "w") as f:
    json.dump(history_data, f, indent=2)

print(f"Hiperparámetros guardados en {history_file}")

Hiperparámetros guardados en best_hyperparams_lstm.json


 Entrenamiento del Modelo Final

In [11]:
# --- Cargar Hiperparámetros y Entrenar Modelo Final ---

# 1. Cargar los mejores hiperparámetros
with open("best_hyperparams_lstm.json", "r") as f:
    history = json.load(f)
best_params = history[-1]["params"]

# 2. Limpiar sesión y fijar semillas para reproducibilidad
tf.keras.backend.clear_session()
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

# 3. Reconstruir el modelo usando FixedTrial (más robusto)
final_model = build_model(
    trial=optuna.trial.FixedTrial(best_params),
    timesteps=X_train.shape[1],
    features=X_train.shape[2]
)
final_model.summary()

# --- Entrenamiento ---
early_stop = EarlyStopping(
    monitor="val_loss", 
    patience=best_params["patience"], 
    restore_best_weights=True, 
    verbose=1)

history = final_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=80,
    batch_size=best_params["batch_size"],
    callbacks=[early_stop],
    verbose=1
)

# Guardar el modelo
final_model.save(f"{MODEL_NAME}.keras")
print(f"Modelo final guardado como {MODEL_NAME}.keras")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 192)               194304    
                                                                 
 dropout (Dropout)           (None, 192)               0         
                                                                 
 dense (Dense)               (None, 192)               37056     
                                                                 
 dropout_1 (Dropout)         (None, 192)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 579       
                                                                 
Total params: 231939 (906.01 KB)
Trainable params: 231939 (906.01 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/80
43/43

In [12]:
# Obtener predicciones
y_val_pred = np.argmax(final_model.predict(X_val), axis=1)
print("LSTM RNN Trend Changes Score:\n", trend_changes_score(y_val, y_val_pred))

10/10 [==============================] - 0s 5ms/step
LSTM RNN Trend Changes Score:
               precision    recall  f1-score   support

       False     0.9340    0.7795    0.8498       254
        True     0.2632    0.5882    0.3636        34

    accuracy                         0.7569       288
   macro avg     0.5986    0.6839    0.6067       288
weighted avg     0.8548    0.7569    0.7924       288



In [13]:
# Reporte completo: precisión, recall y F1 por clase
report = classification_report(y_val, y_val_pred, digits=4)
print("LSTM RNN Report:\n", report)
print("Balanced accuracy:", balanced_accuracy_score(y_val, y_val_pred))

LSTM RNN Report:
               precision    recall  f1-score   support

           0     0.8919    0.7021    0.7857        94
           1     0.2899    0.4878    0.3636        41
           2     0.8562    0.8117    0.8333       154

    accuracy                         0.7301       289
   macro avg     0.6793    0.6672    0.6609       289
weighted avg     0.7874    0.7301    0.7512       289

Balanced accuracy: 0.6672069497705201


In [14]:
# Obtener predicciones
y_pred_test = np.argmax(final_model.predict(X_test), axis=1)
print("LSTM RNN Trend Changes Score:\n", trend_changes_score(y_test, y_pred_test))

10/10 [==============================] - 0s 4ms/step
LSTM RNN Trend Changes Score:
               precision    recall  f1-score   support

       False     0.9200    0.7962    0.8536       260
        True     0.1719    0.3793    0.2366        29

    accuracy                         0.7543       289
   macro avg     0.5459    0.5877    0.5451       289
weighted avg     0.8449    0.7543    0.7917       289



Evaluación y Exportación de Métricas

In [16]:
# --- Evaluación Final y Exportación de Métricas ---

# 2. Calcular reporte de clasificación estándar
report_dict = classification_report(y_val, y_val_pred, output_dict=True, zero_division=0)
precision = report_dict['macro avg']['precision']
recall = report_dict['macro avg']['recall']
f1_score_val = report_dict['macro avg']['f1-score']

# 3. Calcular reporte de cambio de tendencia
report = get_trend_changes_report_dict(y_val, y_val_pred)
trend_change_precision = report['True']['precision']
trend_change_recall = report['True']['recall']
trend_change_f1_score = report['True']['f1-score']

# 4. Organizar métricas
new_metrics = {
    'precision': precision,
    'recall': recall,
    'f1_score': f1_score_val,
    'trend_change_precision': trend_change_precision,
    'trend_change_recall': trend_change_recall,
    'trend_change_f1_score': trend_change_f1_score
}

# 5. Cargar, actualizar y guardar el DataFrame de comparación
output_file = '../../../score_models/model_comparison_metrics.csv'
try:
    comparison_df = pd.read_csv(output_file, index_col='model')
except FileNotFoundError:
    comparison_df = pd.DataFrame([new_metrics], index=[MODEL_NAME])
else:
    comparison_df.loc[MODEL_NAME] = new_metrics

comparison_df.to_csv(output_file, index_label='model')

print(f"Métricas para '{MODEL_NAME}' guardadas en '{output_file}'")
print("\n--- Tabla de Comparación de Modelos ---")
print(comparison_df)

Métricas para 'LSTM_RNN' guardadas en '../../../score_models/model_comparison_metrics.csv'

--- Tabla de Comparación de Modelos ---
                           precision    recall  f1_score  \
model                                                      
lightGBM                    0.929894  0.803173  0.828363   
histGradientBoosting        0.835940  0.732490  0.736232   
Multiclass Neural Network   0.694255  0.672321  0.679897   
Naive Model                 0.820412  0.819136  0.819762   
Random Forest               0.886577  0.659319  0.612598   
Regresion Logistic          0.819553  0.774334  0.783700   
SVM                         0.830711  0.798057  0.806678   
XGBClassifier               0.838116  0.747691  0.760536   
GradientBoosting            0.843002  0.774981  0.786910   
LSTM_RNN                    0.679304  0.667207  0.660895   

                           trend_change_precision  trend_change_recall  \
model                                                                    